In [ ]:
#Predicción de calidad del café

##Requerimientos

In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from openpyxl import load_workbook

print("¡Herramientas listas!")

¡Herramientas listas!


##Carga de datos

In [116]:
# 📂 Leer rutas desde el archivo de texto
with open("rutas_archivos.txt", "r", encoding="utf-8") as f:
    rutas = [line.strip() for line in f if line.strip()]

print(f"Se encontraron {len(rutas)} archivos:")
for ruta in rutas:
    print("•", ruta)

Se encontraron 3 archivos:
• C:\Users\JOSE DANIEL M\OneDrive\Documentos\Universidad\Aprendizaje Estadístico\Tarea 1 Aprendizaje Automatico\CC FT 17   Formato de Control de Calidad Café de Trillado.xlsx
• C:\Users\JOSE DANIEL M\OneDrive\Documentos\Universidad\Aprendizaje Estadístico\Tarea 1 Aprendizaje Automatico\CC FT 18  Formato de  Tostión.xlsx
• C:\Users\JOSE DANIEL M\OneDrive\Documentos\Universidad\Aprendizaje Estadístico\Tarea 1 Aprendizaje Automatico\CC FT 21   Formato de Control de Despachos.xlsx


In [117]:
ruta_trillado, ruta_tostion, ruta_despacho = rutas

In [118]:
# Cargar cada base en su propio DataFrame
df_trillado_1 = pd.read_excel(ruta_trillado, engine="openpyxl", skiprows=8, nrows= 76, header=None,sheet_name="CONTROL CALIDAD CAFE TRILLADO J", dtype={0: str} )
df_trillado_jerico = pd.read_excel(ruta_trillado, engine="openpyxl", skiprows=8,nrows=32,  header=None, sheet_name="Sheet2", dtype={0: str})
df_tostion_1 = pd.read_excel(ruta_tostion, engine="openpyxl", skiprows=6, nrows=499, header=None, sheet_name= "TOSTIÓN JERICÓ L", dtype={0: str})
df_tostion_jerico= pd.read_excel(ruta_tostion, engine="openpyxl", skiprows=6, nrows=30, header=None, sheet_name= "TOSTIÓN JERICÓ", dtype={0: str})
df_despacho_medellin = pd.read_excel(ruta_despacho, engine="openpyxl", skiprows=6, nrows=1583, header=0, sheet_name="TOSTIÓN MEDELLÍN", usecols=[0, 1, 2, 3, 4, 6, 9, 10, 11], dtype={1: str, 2:str})
df_despacho_jerico = pd.read_excel(ruta_despacho, engine="openpyxl", skiprows=6, nrows=27, header=0, sheet_name="TOSTIÓN JERICÓ", usecols=[0, 1, 2, 3, 4, 6, 9, 10, 11], dtype={1: str, 2:str})
df_lotes = pd.read_excel(ruta_trillado, engine="openpyxl", skiprows=0, header=0, sheet_name="LOTES", usecols=[0, 1])

#Etiquetado de columnas manual de Trillado
columnas_trillado = [
    "FECHA",
    "LOTE",
    "MARCA",
    "CANTIDAD",
    "HUMEDAD_%",
    "HUMEDAD_C/NC",
    "MALLAS_#",
    "MALLAS_C/NC",
    "VERIFICACION_FISICA",
    "NOTAS_CATACION",
    "PUNTAJE_#",
    "PUNTAJE_C/NC",
    "LIBERACION_LOTE",
    "RESPONSABLE"
]
columnas_tostion = [
    "FECHA",
    "LOTE",
    "ORIGEN",
    "VARIEDAD",
    "PROCESO",
    "BENEFICIO",
    "PESO_VERDE",
    "MERMA",
    "PESO_TOSTADO",
    "PERFIL",
    "TEMPER_INICIO-FINAL",
    "TIEMPO_TUESTE",
    "OBSERVACIONES",
    "RESPONSABLE"
]
columnas_despacho = [
    "PEDIDO",
    "FECHA_TUESTE",
    "FECHA_EMPAQUE",
    "MARCA",
    "PRESENTACION",
    "CANTIDAD",
    "CLIENTE",
    "RESPONSABLE",
    "VERIFICA"
]
columnas_lotes = [
    "ID",
    "MARCA"
]
df_trillado_1.columns = columnas_trillado
df_trillado_jerico.columns = columnas_trillado
df_tostion_1.columns = columnas_tostion
df_tostion_jerico.columns = columnas_tostion
df_despacho_medellin.columns = columnas_despacho
df_despacho_jerico.columns = columnas_despacho
df_lotes.columns = columnas_lotes
print("Archivos cargados correctamente.")

Archivos cargados correctamente.


Haremos join de las bases de datos tal que quede solo una de trillado, una de tostión y una de despacho.

In [119]:
# Mantengamos un atributo de la sede solo por orden
df_trillado_1['SEDE'] = 'medellin'
df_trillado_jerico['SEDE'] = 'jerico'
# y luego les hacemos join
df_trillado = pd.concat([df_trillado_1, df_trillado_jerico], ignore_index=True)

df_tostion_1['SEDE'] = 'medellin'
df_tostion_jerico['SEDE'] = 'jerico'
df_tostion = pd.concat([df_tostion_1, df_tostion_jerico], ignore_index=True)

df_despacho_medellin['SEDE'] = 'medellin'
df_despacho_jerico['SEDE'] = 'jerico'
df_despacho = pd.concat([df_despacho_medellin, df_despacho_jerico], ignore_index=True)  
print("Bases de trillado, tostión y despacho unidas correctamente.")

Bases de trillado, tostión y despacho unidas correctamente.


In [81]:
#Eliminamos los dataframes que ya no necesitamos
del df_trillado_1, df_trillado_jerico, df_tostion_1, df_tostion_jerico, df_despacho_medellin, df_despacho_jerico

In [120]:
df_trillado

,FECHA,LOTE,MARCA,CANTIDAD,HUMEDAD_%,HUMEDAD_C/NC,MALLAS_#,MALLAS_C/NC,VERIFICACION_FISICA,NOTAS_CATACION,PUNTAJE_#,PUNTAJE_C/NC,LIBERACION_LOTE,RESPONSABLE,SEDE
0,19-07-22,01-190722,Madre Laura,765.00,10.9,C,14,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,Si,LFQ,medellin
1,19-7-22,09-190722,Tabi Natural,204.00,10.2,C,14,C,C,"Frutas maduras, nibs de cacao, acidez brillant...",85,C,Si,LFQ,medellin
2,19-07-22,10-190722,Don Mario,165.00,10.7,C,14,C,C,"Panela, durazno, miel, acidez brillante citric...",84.5,C,Si,LFQ,medellin
3,27-07-22,07-19-07-22,Don Felix,0.45,10.5,C,14,C,C,"Moras maduras, chocolate negro, acidez media c...",84.5,C,Si,LFQ,medellin
4,31-10-22,01-291022,Madre Laura,105.00,10.7,C,14,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,Si,LFQ,medellin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,2023-10-27 00:00:00,01-271023,Madre Laura,20.00,10.5,C,14,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84.0,C,SI,AC,jerico
104,10-Ene-2024,01-100124,Madre Laura,20.00,10.4,c,14,c,c,"Chocolate negro, toque frutal, cuerpo medio, a...",84.0,C,SI,AC,jerico
105,02-Abril-2024,01-020424,Madre Laura,20.00,10.5,C,14,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84.0,C,SI,AC,jerico
106,2024-06-20 00:00:00,01-200624,Madre Laura,14.00,10.8,C,14,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84.0,C,SI,AC,jerico


In [121]:
df_trillado.iloc[:, 0].apply(type).value_counts()

FECHA
<class 'str'>    108
Name: count, dtype: int64

Hagamos el ejercicio de ver de qué tipos está recibiendo los dataframes la información

In [122]:
def clasificar_variables(df):
    resumen = {}

    for col in df.columns:
        tipo = df[col].dtype

        if pd.api.types.is_numeric_dtype(df[col]):
            resumen[col] = 'numérica'
        elif pd.api.types.is_bool_dtype(df[col]):
            resumen[col] = 'booleana'
        elif isinstance(tipo, pd.CategoricalDtype):
            resumen[col] = 'categórica'
        elif df[col].apply(type).eq(str).all():
            resumen[col] = 'string (pura)'
        elif pd.api.types.is_string_dtype(df[col]):
            resumen[col] = 'string (nativa)'
        elif pd.api.types.is_datetime64_any_dtype(df[col]):
            resumen[col] = 'fecha'
        else:
            resumen[col] = 'otros'

    return pd.Series(resumen, name='tipo_variable')


In [123]:
clasificar_variables(df_tostion)

FECHA                  string (pura)
LOTE                   string (pura)
ORIGEN                 string (pura)
VARIEDAD               string (pura)
PROCESO                string (pura)
BENEFICIO              string (pura)
PESO_VERDE                  numérica
MERMA                       numérica
PESO_TOSTADO                numérica
PERFIL                 string (pura)
TEMPER_INICIO-FINAL    string (pura)
TIEMPO_TUESTE                  otros
OBSERVACIONES                  otros
RESPONSABLE            string (pura)
SEDE                   string (pura)
Name: tipo_variable, dtype: object

In [124]:
clasificar_variables(df_trillado)

FECHA                  string (pura)
LOTE                   string (pura)
MARCA                  string (pura)
CANTIDAD                    numérica
HUMEDAD_%                      otros
HUMEDAD_C/NC           string (pura)
MALLAS_#                    numérica
MALLAS_C/NC            string (pura)
VERIFICACION_FISICA    string (pura)
NOTAS_CATACION                 otros
PUNTAJE_#                      otros
PUNTAJE_C/NC                   otros
LIBERACION_LOTE                otros
RESPONSABLE                    otros
SEDE                   string (pura)
Name: tipo_variable, dtype: object

In [125]:
clasificar_variables(df_despacho)

PEDIDO                   otros
FECHA_TUESTE     string (pura)
FECHA_EMPAQUE    string (pura)
MARCA            string (pura)
PRESENTACION     string (pura)
CANTIDAD              numérica
CLIENTE          string (pura)
RESPONSABLE      string (pura)
VERIFICA         string (pura)
SEDE             string (pura)
Name: tipo_variable, dtype: object

Vamos primero con recibir todas las fechas como datetime

In [126]:
# Cambiar a mano los registros más problemáticos
df_trillado.iloc[104, 0] = "10-01-2024"
df_trillado.iloc[105, 0] = "02-04-2024"

In [127]:
df_trillado['FECHA'] = pd.to_datetime(df_trillado['FECHA'], format='mixed', dayfirst=True, errors='coerce')
df_tostion['FECHA'] = pd.to_datetime(df_tostion['FECHA'], format='mixed', dayfirst=True, errors='coerce')
df_despacho['FECHA_TUESTE'] = pd.to_datetime(df_despacho['FECHA_TUESTE'], format='mixed', dayfirst=True, errors='coerce')
df_despacho['FECHA_EMPAQUE'] = pd.to_datetime(df_despacho['FECHA_EMPAQUE'], format='mixed', dayfirst=True, errors='coerce')

Ahora separemos en varias columnas las variables que llevan más de un dato en cada una, o llevan unidades. Ellas son: TIEMPO_TUESTE que lleva el tiempo en minutos pero en el formato "minutos:segundos",
TEMPER_INICIO-FINAL que lleva dos temperaturas,PRESENTACION que va acompañado de las unidades y LOTE en la tabla de Trillado que es N° tipo de café + fecha de trilla(Día, mes y año). Este tercero es más complejo, habrán más detalles en el código específico.

In [128]:
# Función para convertir el tiempo en formato "minutos:segundos" a segundos
def convertir_tiempo_a_segundos(serie):
    """
    Convierte una serie de strings como '5:30', '10:15', etc.
    en su valor numérico en segundos como entero.
    """
    def convertir(valor):
        try:
            if pd.isna(valor):
                return None
            partes = str(valor).strip().split(':')
            minutos = int(partes[0])
            segundos = int(partes[1]) if len(partes) > 1 else 0
            return minutos * 60 + segundos
        except:
            return None

    return serie.apply(convertir)

In [129]:
# Aplicar la conversión a la columna TIEMPO_TUESTE
df_tostion['TIEMPO_TUESTE'] = convertir_tiempo_a_segundos(df_tostion['TIEMPO_TUESTE'])

In [130]:
# Función para normalizar a gramos en PRESENTACION
def normalizar_presentacion_a_gramos(serie):
    """
    Convierte una serie de strings como '250 Gr', '0.5 KG', '20kg', etc.
    en su valor numérico en gramos como entero.
    """
    def convertir(valor):
        try:
            val = str(valor).strip().lower()
            if 'kg' in val:
                num = float(val.split()[0].replace('kg', '').strip())
                return int(num * 1000)
            elif 'gr' in val:
                num = float(val.split()[0].replace('gr', '').strip())
                return int(num)
            else:
                num = float(val.split()[0])
                return int(num)
        except:
            return None  

    return serie.apply(convertir)

In [131]:
# Aplicar la normalización a la columna PRESENTACION
df_despacho['PRESENTACION'] = normalizar_presentacion_a_gramos(df_despacho['PRESENTACION'])

In [132]:
#Separar la columna TEMPER_INICIO-FINAL en dos columnas TEM_INICIO y TEM_FINAL
df_tostion[['TEM_INICIO', 'TEM_FINAL']] = df_tostion['TEMPER_INICIO-FINAL'].str.replace('°', '', regex=False).str.split('/', expand=True)

El problema de la columna LOTE. Esta columna tiene se crea al unir el id de la marca con la fecha de trillado en ddmmaa. Pero resulta muy útil porque la columna de MARCA tiene varias insconsistencias con nombres distintos para una misma marca. Por tanto se desprenderá de la información de la columna MARCA y se reemplazará con lo que dice en la columna de LOTE.

In [133]:
mapa_marcas = df_lotes.set_index('ID')['MARCA'].to_dict()

# Crea la nueva marca solo si hay LOTE
nueva_marca = (
    df_trillado['LOTE']
    .astype(str)
    .where(df_trillado['LOTE'].notna())        # solo si no es NaN
    .str.split('-').str[0]
    .dropna()
    .astype(int)
    .map(mapa_marcas)
)

# Reemplaza MARCA solo donde el LOTE no es NaN
df_trillado['MARCA'] = df_trillado['LOTE'].notna().map(lambda x: True if x else False) * nueva_marca.combine_first(df_trillado['MARCA'])

In [134]:
def extraer_marca_desde_lote(lote):
    try:
        if pd.notna(lote):
            id_marca = int(str(lote).split('-')[0])
            return mapa_marcas.get(id_marca, None)
    except:
        return None

df_tostion['MARCA'] = df_tostion['LOTE'].apply(extraer_marca_desde_lote)

Pasemos a numérico todos los datos que deberían serlo, pero se están recibiendo mal

In [135]:
print(df_trillado.columns.tolist())
print(df_tostion.columns.tolist())
print(df_despacho.columns.tolist())

['FECHA', 'LOTE', 'MARCA', 'CANTIDAD', 'HUMEDAD_%', 'HUMEDAD_C/NC', 'MALLAS_#', 'MALLAS_C/NC', 'VERIFICACION_FISICA', 'NOTAS_CATACION', 'PUNTAJE_#', 'PUNTAJE_C/NC', 'LIBERACION_LOTE', 'RESPONSABLE', 'SEDE']
['FECHA', 'LOTE', 'ORIGEN', 'VARIEDAD', 'PROCESO', 'BENEFICIO', 'PESO_VERDE', 'MERMA', 'PESO_TOSTADO', 'PERFIL', 'TEMPER_INICIO-FINAL', 'TIEMPO_TUESTE', 'OBSERVACIONES', 'RESPONSABLE', 'SEDE', 'TEM_INICIO', 'TEM_FINAL', 'MARCA']
['PEDIDO', 'FECHA_TUESTE', 'FECHA_EMPAQUE', 'MARCA', 'PRESENTACION', 'CANTIDAD', 'CLIENTE', 'RESPONSABLE', 'VERIFICA', 'SEDE']


In [136]:
# Convertir columnas numéricas a tipo numérico
def convertir_a_numerico(df, columnas):
    for col in columnas:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df
# Listas de columnas a convertir
columnas_trillado_num = ['CANTIDAD', 'HUMEDAD_%', 'MALLAS_#', 'PUNTAJE_#']
columnas_tostion_num = ['PESO_VERDE', 'MERMA', 'PESO_TOSTADO', 'TEM_INICIO', 'TEM_FINAL', 'TIEMPO_TUESTE']
columnas_despacho_num = ['PEDIDO', 'PRESENTACION', 'CANTIDAD']

# Aplicar la conversión
df_trillado = convertir_a_numerico(df_trillado, columnas_trillado_num)  
df_tostion = convertir_a_numerico(df_tostion, columnas_tostion_num)
df_despacho = convertir_a_numerico(df_despacho, columnas_despacho_num)

In [ ]:
# (Opcional) Pasar a tipo  todas las columnas que son de C o NC
columnas_trillado_boo = ['HUMEDAD_C/NC', 'MALLAS_C/NC', 'PUNTAJE_C/NC']

for col in columnas_trillado_boo:
    df_trillado[col] = df_trillado[col].str.upper().map({'C': True, 'NC': False})
   

In [145]:
 df_trillado['LIBERACION_LOTE'] = df_trillado['LIBERACION_LOTE'].str.upper().map({'SI': True, 'NO': False})

In [148]:
for col in columnas_trillado_boo:
    df_tostion[0, col] = True

In [149]:
df_trillado

,FECHA,LOTE,MARCA,CANTIDAD,HUMEDAD_%,HUMEDAD_C/NC,MALLAS_#,MALLAS_C/NC,VERIFICACION_FISICA,NOTAS_CATACION,PUNTAJE_#,PUNTAJE_C/NC,LIBERACION_LOTE,RESPONSABLE,SEDE
0,2022-07-19,01-190722,Madre Laura Lavado,765.00,10.9,NaN,14,NaN,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84.0,NaN,NaN,LFQ,medellin
1,2022-07-19,09-190722,Don Juan Tabi,204.00,10.2,True,14,True,C,"Frutas maduras, nibs de cacao, acidez brillant...",85.0,True,NaN,LFQ,medellin
2,2022-07-19,10-190722,Doña Liceth,165.00,10.7,True,14,True,C,"Panela, durazno, miel, acidez brillante citric...",84.5,True,NaN,LFQ,medellin
3,2022-07-27,07-19-07-22,Don Felix,0.45,10.5,True,14,True,C,"Moras maduras, chocolate negro, acidez media c...",84.5,True,NaN,LFQ,medellin
4,2022-10-31,01-291022,Madre Laura Lavado,105.00,10.7,True,14,True,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84.0,NaN,NaN,LFQ,medellin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,2023-10-27,01-271023,Madre Laura Lavado,20.00,10.5,True,14,True,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84.0,True,True,AC,jerico
104,2024-01-10,01-100124,Madre Laura Lavado,20.00,10.4,True,14,True,c,"Chocolate negro, toque frutal, cuerpo medio, a...",84.0,True,True,AC,jerico
105,2024-04-02,01-020424,Madre Laura Lavado,20.00,10.5,True,14,True,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84.0,True,True,AC,jerico
106,2024-06-20,01-200624,Madre Laura Lavado,14.00,10.8,True,14,True,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84.0,True,True,AC,jerico
